# A chemical lighthouse shadowing model of protoplanetary disks
## By Jorge Pérez González

This notebook collects the code described in the MSci thesis: **"Chemical beacons in planet formation.
Can shadowing effects permanently alter the gas phase C/O ratio?"**

For any questions please email me via zcapere@ucl.ac.uk

In [ ]:
import grid_functions as gs
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

plt.rc('axes', linewidth=3)
plt.rc('xtick.minor', size=4, width=2)
plt.rc('xtick.major', size=8, width=2)
plt.rc('ytick.minor', size=4, width=2)
plt.rc('ytick.major', size=8, width=2)

## 1. Model Inputs. Disc parameter prescription
This section involves setting up the disc structure. For details regarding the parameters please see comments embeded in code.

In [ ]:
##########
# INPUTS #
##########

structure_path = '/Users/jorgeperezgonzalez/Documents/UCL/UCL_4_Fourth_Year_23_24/MSci_Project/Code/Structure_Arrays/'
plot_path      = '/Users/jorgeperezgonzalez/Documents/UCL/UCL_4_Fourth_Year_23_24/MSci_Project/Code/Grid_Plots/'

# Constants
AUCM        = 1.49e13      # conversion factor between au and cm
uMCM        = 1e-4         # conversion factor between micro-m and cm
SBconst     = 5.67051e-5   # stefan - boltmann constant in erg cm-2 K-4 s-1
Msol        = 1.989e+33    # solar mass in grams

# Physical parameters
alpha       = 1e-3         # Shakura-Sunyaev alpha parameter parametrizing the viscosity
R_c         = 50           # radius in AU to define surface density
Sigma_c     = 82.75        # Surface density at radius R_c g/cm2
exp_density = 1            # scaling order in surface density profile
h_c         = 0.1          # Scale height at radius R_c. In AU
psi         = 0.2          # exponent to define h(r)
gtd_ratio   = 100          # gas to dust ratio. For interstellar 100
tmid1       = 200          # midplane temperature at 1 AU
tatm1       = 1000         # atmosphere temperature at 1 AU
delta       = 2            # steepness of profile parameter 1
q           = 0.55         # steepness of profile parameter 2
Mstar       = 2.66 * Msol  # star mass in solar masses





#########################
# SETUP DUST GRAIN BINS #
#########################

# The grain size bins are set as follows:
# 1. Start with a minimum grain size
# 2. Choose the number of grain sizes you want ('n_bins')
# 3. Each bin is twice the mass of the previous, so the more bins, the larger the mass of the largest grain
# 4. The rest of the code is set up to automatically work, no matter how many bin sizes are chosen
# 5. It also prints a list all grain masses and radii for easy reference
# 6. I use 54 bins which results in a max grain size of ~ 1mm

# INPUTS ------------------------------------------------
r_grain_min    = 0.005  # minimum grain size in microns
rho_grain      = 2      # grain density in g/cm^3
# -------------------------------------------------------

mass_grain_min = ((4/3) * np.pi * (r_grain_min*uMCM)**3) * rho_grain 

n_bins = 54
r_bins = np.zeros(shape=(n_bins))   # an array containing the radius values for each grain size bin
m_bins = np.zeros(shape=(n_bins))   # an array containing the mass values for each grain size bin

# A loop to calculate grain sizes and masses to be used in the grain size distribution
size_factor = 1
for i in range(0, n_bins):
    mass_grain = mass_grain_min * size_factor
    r_grain    = (mass_grain / ((4/3)*np.pi*rho_grain))**(1/3)
    print(f'Mass: {mass_grain:.2e}    Radius: {r_grain/uMCM:.3f}')
    m_bins[i] = mass_grain
    r_bins[i] = r_grain
    size_factor = size_factor*2

In [ ]:
###################
# SETUP DENSITIES #
###################

# Surface density (see equations in report)

# DISC GRID SETUP ----------------------------------------------------------------------------
R = np.logspace(0.1, 2.5, 100)
z_arrays = []
r_arrays = np.tile(R,(50,1))
for i in range(100):
    z_arrays.append(np.linspace(0, R[i]/2, 50))
# --------------------------------------------------------------------------------------------



# SURFACE DENSITY STRUCTURE SETUP ----------------------------------------------------------------------------
h = gs.scale_height(R, R_c, h_c, psi)                                  # radially varying scale height
hdot = h_c/R_c*psi*(R/R_c)**(psi-1)                                    # dh/dr 
phi = R * hdot                                                         # angle from disc midplane to scale height
surface_density_gas  = gs.density_radial(R, R_c, Sigma_c, exp_density) # gas surface density
surface_density_dust = surface_density_gas/gtd_ratio                   # dust surface density

np.save(structure_path+'surface_density_gas', surface_density_gas)
np.save(structure_path+'surface_density_dust', surface_density_dust)
# -------------------------------------------------------------------------------------------------------------



# VERTICAL DENSITY STRUCTURE SETUP ----------------------------------------------------------------------------
gas_density  = np.zeros((50,100))
dust_density = np.zeros((50,100, n_bins))
n_dust       = np.zeros((50,100, n_bins))


# Gas density (rho)
for i in range(50):
    for j in range(100):
        cell_density_gas = gs.density_vertical_gas(surface_density_gas[j], z_arrays[j][i]/R[j], h[j], R[j]*AUCM)
        gas_density[i,j] = cell_density_gas
print("Gas Density Set")


# Calculate chi values for dust
omegas = gs.Omega(Mstar, R*AUCM)
chis   = np.zeros((50, 100, n_bins))
for i in range(50):
    for j in range(100):
        for k in range(n_bins):
            t_fric = (rho_grain/gas_density[i][j]) * (r_bins[k]*uMCM)
            chi = ((1+ ((omegas[j]*t_fric)/0.001))*((1+2*omegas[j]*t_fric)/(1+omegas[j]*t_fric))) ** (-0.5)
            chis[i][j][k] = chi


# Calculate surface density per grain size bin
surf_dens_per_bin = np.zeros(shape=(100, n_bins))               # I was a bit confused by how your code does this... changed it just for my own understanding
for r in range(100):
    n = np.array(r_bins)**-3.5
    mass_in_bin_unscaled = np.zeros(shape=(n_bins))
    
    for i in range(n_bins):
        column_mass = surface_density_dust[r]
        mass_in_bin_unscaled[i] = n[i] * m_bins[i]

    total_mass_unnorm = np.sum(mass_in_bin_unscaled)
    norm_factor       = column_mass/total_mass_unnorm
    mass_in_bin       = mass_in_bin_unscaled * norm_factor
    
    surf_dens_per_bin[r] = mass_in_bin
    

# Calculate f values
f_frac = np.zeros(shape=(100, n_bins))
for r in range(100):
    for m in range(n_bins):
        frac = surf_dens_per_bin[r][m] / sum(surf_dens_per_bin[r])
        f_frac[r][m] = frac
        

# Dust density (rho)
for i in range(50):
    for j in range(100):
        for k in range(n_bins):
            cell_density_dust = gs.density_vertical_big(f_frac[j][k], surface_density_dust[j], z_arrays[j][i]/R[j], h[j], chis[i][j][k], R[j]*AUCM)
            dust_density[i,j,k] = cell_density_dust


# Dust number density
for i in range(50):
    for j in range(100):
        for k in range(n_bins):
            n_dust[i][j][k] = dust_density[i][j][k] / m_bins[k]            
            

print("Dust Density Set")

np.save(structure_path+'h', h)
np.save(structure_path+'chis', chis)
np.save(structure_path+'dust_density', dust_density)
np.save(structure_path+'gas_density', gas_density)
np.save(structure_path+'n_dust', n_dust)
# ----------------------------------------------------------------------------------------------------------------

In [ ]:
###############
# TEMPERATURE #
###############
           
# Temperature array
temp = np.zeros(shape=(50, 100))  # set up an empty array to store temperature values


# Constants (SI)
k      = 1.380649e-23
G      = 6.67430e-11
m_star = 1e30 * 2.5
mu     = 2.3
m_H    = 1.6735575e-27

# Calculate temperature values
for z in range(50):
    for r in range(100):
        
        r_au = R[r]
        z_au = z_arrays[r][z]
        
        tmid = tmid1 * (r_au**-q)
        tatm = tatm1 * (r_au**-q)
        
        # Scale height
        h = h_c*((r_au/R_c)**psi)
        H = r_au*h
        zq = 4 * H

        # Temperature
        if z_au < zq:
            temp[z,r] = tmid + ((tatm-tmid) * ((np.sin((np.pi*z_au)/(2*zq)))**(2*delta)))
        else:
            temp[z,r] = tatm

np.save(structure_path+'temperature', temp)
print("Temperature Structure Set")


############################
# SNOWLINE MASK
############################
snowline_permanent = np.load(structure_path+ 'zr_ice.npy')
snowline_temporary = np.load(structure_path+ 'zr_shadow.npy')

snowline_mask = np.zeros((50,100), dtype = "int")
import matplotlib.colors
for i in range(50):
    for j in range(100):
        shadow = int(snowline_temporary[j])
        ice = int(snowline_permanent[j])
        if (i >= shadow) & (i <=ice):
            snowline_mask[i,j] = 1
print("Snowlines included")            

In [ ]:
###########################
# PLOT DENSITIES TO CHECK #
###########################
dust_density[dust_density < 1e-30] = 1e-30
n_dust[n_dust < 1e-30] = 1e-30
gas_ndensity = gas_density/1.6735e-24

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 8), sharey=True,sharex=True)

rcoord = np.logspace(np.log10(1), 2.5, num=100)
zrcoord = np.linspace(0,.5, 50)
r, zr  = np.meshgrid(rcoord, zrcoord)

levels_rho = np.arange(-20, -8, 1)
levels_n   = np.arange(-8, 5, 1)
levels_ngas   = np.arange(2, 14, 1)

rho_gas_plot  = ax[0,0].contourf(r, zr, np.log10(gas_density), levels=levels_rho, cmap='inferno', extend='both')
rho_dust_plot = ax[0,1].contourf(r, zr, np.log10(np.sum(dust_density, axis=2)), levels=levels_rho, cmap='inferno', extend='both')
n_gas_plot    = ax[1,0].contourf(r, zr, np.log10(gas_ndensity), levels=levels_ngas, cmap='viridis', extend='both')
n_dust_plot   = ax[1,1].contourf(r, zr, np.log10(n_dust[:,:,9]), levels=levels_n, cmap='viridis', extend='both')

cb1 = fig.colorbar(rho_gas_plot, ax=ax[0,0], ticks=levels_rho)
cb2 = fig.colorbar(rho_dust_plot, ax=ax[0,1], ticks=levels_rho)
cb3 = fig.colorbar(n_gas_plot, ax=ax[1,0], ticks=levels_ngas)
cb4 = fig.colorbar(n_dust_plot, ax=ax[1,1], ticks=levels_n)

ax[0,0].annotate('$\\rho_\\mathrm{gas}$', (2, 0.4), c='white', fontsize=22)
ax[0,1].annotate('$\\rho_\\mathrm{dust}$', (2, 0.4), c='white', fontsize=22)
ax[1,0].annotate('n$_\\mathrm{gas}$', (2, 0.4), c='white', fontsize=22)
ax[1,1].annotate('n$_\\mathrm{dust}$', (2, 0.4), c='white', fontsize=22)

for i in range(2):
    for j in range(2):
        ax[i,j].set_xscale('log')
        ax[i,j].tick_params(axis='x', labelsize=12)
        ax[i,j].tick_params(axis='y', labelsize=12)
ax[0,0].set_ylabel('z/r', fontsize=15)
ax[1,0].set_ylabel('z/r', fontsize=15)
ax[1,0].set_xlabel('r (AU)', fontsize=15)
ax[1,1].set_xlabel('r (AU)', fontsize=15)

plt.tight_layout()
plt.savefig( f"{plot_path}density_structure.pdf", bbox_inches='tight')
plt.show()

In [ ]:
############################
# PLOT TEMPERATURE         #
############################

temp_shadow = temp - 50
temp_shadow[temp_shadow<50] = 1

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6), sharey=True,sharex=True)

levels = [np.log10(10), np.log10(20), np.log10(30), np.log10(40), np.log10(50), np.log10(60), np.log10(70),
          np.log10(80), np.log10(90), np.log10(100), np.log10(200), np.log10(300), np.log10(500), np.log10(1000)]
ticks_str_tdust = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100', '200', '300', '500', '1000']
ticks = levels

temp_plt = ax[0].contourf(r, zr, np.log10(temp), extend='both', cmap="inferno", levels=levels)
temp_plt_shadow = ax[1].contourf(r, zr, np.log10(temp_shadow), extend='both', cmap="inferno", levels=levels)
cbar_ax = fig.add_axes([0.05, -0.05, .95, 0.05])  # [left, bottom, width, height]
cbar = fig.colorbar(temp_plt, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r'Temperature (K)',fontsize=18)
cbar.set_ticklabels(ticks_str_tdust,fontsize=15)
cbar.set_ticks(ticks)

ax[0].annotate('$T_\\mathrm{dust}$ (K)', (2, 0.4), c='black', fontsize=22)
ax[1].annotate('$T_\\mathrm{shadow}$ (K)', (2, 0.4), c='black', fontsize=22)

for i in range(2):
    ax[i].set_xscale('log')
    ax[i].tick_params(axis='x', labelsize=15)
    ax[i].tick_params(axis='y', labelsize=15)
    ax[i].set_xlabel('r (AU)', fontsize=15)
ax[0].set_ylabel('z/r', fontsize=15)

ax[0].set_xlabel('r (AU)', fontsize=15)

    
plt.tight_layout()
plt.savefig( f"{plot_path}temperature_structure.pdf", bbox_inches='tight')
plt.show()

In [ ]:
############################
# # INTERSTELLAR RADIATION FIELD    
############################

g0 = np.load('/Users/jorgeperezgonzalez/Documents/UCL/UCL_4_Fourth_Year_23_24/MSci_Project/Code/Structure_Arrays/g0_dali_regrid.npy')


fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6), sharey=True,sharex=True)

levels = np.arange(-10, 10, 1)

g0_plt = ax[0].contourf(r, zr, np.log10(g0), extend='both', cmap="viridis", levels=levels)
mask = ax[1].contourf(r, zr, snowline_mask, extend='both', cmap="coolwarm")


cbar_ax = fig.add_axes([0.05, -0.05, .95, 0.05])  # [left, bottom, width, height]
cbar = fig.colorbar(g0_plt, cax=cbar_ax, orientation='horizontal')
cbar.set_label('log($G_0)$ (Draine Fields)',fontsize=18)
cbar.set_ticklabels(levels, fontsize = 15)

ax[0].annotate('log($G_0)$', (2, 0.4), c='black', fontsize=22)
ax[1].annotate('Snowline Mask', (2, 0.4), c='white', fontsize=22)

for i in range(2):
    ax[i].set_xscale('log')
    ax[i].tick_params(axis='x', labelsize=18)
    ax[i].tick_params(axis='y', labelsize=18)
    ax[i].set_xlabel('r (AU)', fontsize=18)
ax[0].set_ylabel('z/r', fontsize=18)

ax[0].set_xlabel('r (AU)', fontsize=18)

    
plt.tight_layout()
plt.savefig( f"{plot_path}desoption_structure.pdf", bbox_inches='tight')
plt.show()

In [ ]:
########################
# WATER AND CO2 ABUNDANCES
########################

water_density = np.load('/Users/jorgeperezgonzalez/Documents/UCL/UCL_4_Fourth_Year_23_24/MSci_Project/Code/Structure_Arrays/water_density.npy')
co2_density = .5 * water_density


rcoord = np.logspace(np.log10(1), 2.5, num=100)
zrcoord = np.linspace(0,.5, 50)
r, zr  = np.meshgrid(rcoord, zrcoord)
levels_molec = np.arange(-5, 10, 1)


fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6), sharey=True,sharex=True)


h20_plot  = ax[0].contourf(r, zr, np.log10(water_density), levels=levels_molec, cmap='viridis', extend='both')
c02_plot = ax[1].contourf(r, zr, np.log10(co2_density), levels=levels_molec, cmap='viridis', extend='both')


cbar_ax = fig.add_axes([0.05, -0.05, .95, 0.05])  # [left, bottom, width, height]
cbar = fig.colorbar(h20_plot, cax=cbar_ax, orientation='horizontal')
cbar.set_label('log(X) (cm$^{-3}$)',fontsize=18)
cbar.set_ticklabels(levels_molec, fontsize = 15)

ax[0].annotate('$\\rho_\mathrm{H_2O}$', (2, 0.4), c='white', fontsize=22)
ax[1].annotate('$\\rho_\mathrm{CX}$', (2, 0.4), c='white', fontsize=22)

for i in range(2):
    ax[i].set_xscale('log')
    ax[i].tick_params(axis='x', labelsize=18)
    ax[i].tick_params(axis='y', labelsize=18)
    ax[i].set_xlabel('r (AU)', fontsize=18)
ax[0].set_ylabel('z/r', fontsize=18)

ax[0].set_xlabel('r (AU)', fontsize=18)

    
plt.tight_layout()
plt.savefig( f"{plot_path}chemistry.pdf", bbox_inches='tight')
plt.show()

## 2. Dust Evolution. Dust Growth and Settling.
This section involves the dust evolution component of the model and its settling part. For details regarding the parameters please see comments embeded in code.

In [ ]:
############################
# DUST GROWTH AND SETTLING #
############################

# Inputs
h                    = np.load(structure_path+'h.npy')
chis                 = np.load(structure_path+'chis.npy')
surface_density_dust = np.load(structure_path+'surface_density_dust.npy')
dust_density         = np.load(structure_path+'dust_density.npy')
gas_density          = np.load(structure_path+'gas_density.npy')
n_density_initial    = np.load(structure_path+'n_dust.npy')
temperature          = np.load(structure_path+'temperature.npy')



# Constants
G     = 6.67e-8          # cgs
k_B   = 1.380649e-16     # cgs
yrSec = 3.156e+7

# Setup arrays
rcoord = np.logspace(np.log10(1), 2.5, num=100)
zrcoord = np.linspace(0,.5, 50)
r, zr  = np.meshgrid(rcoord, zrcoord)

R = np.logspace(0.1, 2.5, 100)
dR = np.diff(R)
dR = np.concatenate((np.array([10**0.1]), dR), axis=0)

z_arrays = []
dZ = np.zeros(100)
for i in range(100):
    z_arrays.append(np.linspace(0, R[i]/2, 50))
    dZ[i] = (R[i]/2)/100

# Timesteps
# - Here as an example I run for 1 transit = 10 years, for 100 orbits (1000 years total)... takes ~2 mins to run
t       = yrSec * 10000 # n_orbits
dt      = yrSec * 10 # n_years
n_steps = int(t/dt)

n_densities_evolution        = np.zeros((50, 100, n_bins))
n_densities_evolution[:,:,:] = n_density_initial 


# Total mass in between-shadow region
step_masses = np.zeros(n_steps)
step_masses[0] = np.sum(n_density_initial[snowline_mask] * m_bins) 

step_areas = np.zeros(n_steps)
step_areas[0] = np.sum(n_density_initial[snowline_mask] * 4 * np.pi * r_bins)

for step in range(n_steps-1):
    print(f'STEP {step+1}/{n_steps-1}')
    
    ##########
    # GROWTH #
    ##########
            
    for i in (range(50)):
        for j in range(100):
            for k in range(n_bins-1):

                grain_mass    = m_bins[k]
                velocity      = np.sqrt(3 * k_B * temp[i,j] / grain_mass)               # cm/s
                cross_section = np.pi * ((r_bins[k]) ** 2 )                                # cm^2
                rate          = n_densities_evolution[i,j,k] * velocity * cross_section   # 1/sec
                
                n_collisions  = rate * dt
                
                if n_collisions < (0.5 * n_densities_evolution[i,j,k]):
                    n_densities_evolution[i,j,k+1] = n_densities_evolution[i,j,k+1] + n_collisions
                    n_densities_evolution[i,j,k]   = n_densities_evolution[i,j,k] - (2*n_collisions)
                else:
                    n_densities_evolution[i,j,k+1] = n_densities_evolution[i,j,k+1] + ((n_densities_evolution[i,j,k]/2))
                    n_densities_evolution[i,j,k]   = 0
                    
                    
    ############
    # SETTLING #
    ############
    # Calculate surface density per grain size bin
    surf_dens_per_bin = np.zeros(shape=(100, n_bins)) 
    
    for k in range(n_bins):
        rho_dust = n_densities_evolution[:,:,k] * m_bins[k]
        for j in range(100):
            column_density = np.trapz(rho_dust[:,j], x = z_arrays[j]*AUCM)
            surf_dens_per_bin[j,k] = column_density
    

    # Calculate f values
    f_frac = np.zeros(shape=(100, n_bins))
    for r in range(100):
        for m in range(n_bins):
            frac = surf_dens_per_bin[r][m] / sum(surf_dens_per_bin[r])
            f_frac[r][m] = frac
        

        # Dust density (rho)
    for i in range(50):
        for j in range(100):
            for k in range(n_bins):
                cell_density_dust = gs.density_vertical_big(f_frac[j][k], surface_density_dust[j], z_arrays[j][i]/R[j], h[j], chis[i][j][k], R[j]*AUCM)
                dust_density[i,j,k] = cell_density_dust
    
    for k in range(n_bins):
        n_densities_evolution[:,:,k] = dust_density[:,:,k] / m_bins[k]
    ###############################
    # TOTAL MASS IN-BETWEEN REGIONS
    ###############################
    total_mass_region = 0
    total_surface_area = 0
    for i in range(50):
        for j in range(100):
            if snowline_mask[i,j] == 1:
                for k in range(n_bins):
                    n_grains = n_densities_evolution[i,j,k] * 2 * np.pi * R[j] * (1/6) * dR[j] * dZ[j]  * (AUCM ** 3)
                    total_mass_region += n_grains * m_bins[k]
                    total_surface_area += (n_grains * 4 * np.pi * r_bins[k])
    step_masses[step] = total_mass_region
    step_areas[step] = total_surface_area
print('**FINISHED**')

Below we plot the evolution of our size distribution and the surface area lost per time step.

In [ ]:
###########################################################
# SIZE DISTRIBUTION                                       # 
###########################################################

size_dist_original = np.sum(n_density_initial[:,:,:], axis =(0,1))
size_dist_evolved = np.sum(n_densities_evolution[:,:,:], axis =(0,1))

plt.figure(dpi = 100)
plt.plot(r_bins, size_dist_original, linewidth = 3, label = "Start", c = "k")
plt.plot(r_bins, size_dist_evolved, linewidth = 3, label = "End", c = "tab:blue")
plt.xscale("log")
plt.yscale("log")
plt.ylabel(r"Number density (cm$^{-3}$)")
plt.xlabel(r" Grain Size (cm)")
plt.ylim(1e-16,1e10)
plt.legend(loc='best', frameon=False)
plt.savefig(plot_path + "size_distribution.pdf",bbox_inches='tight')
plt.show()

In [ ]:
###############################
# SURFACE AREA LOST
###############################

# 1. Cummulative studies ------------------------------------------------------
M_lost              = np.diff(step_masses[:997])
cumulative_lost     = np.zeros(997)
N_mmgrain_cum       = np.zeros(997)
surface_area_cum    = np.zeros(997)
fractional_area_cum = np.zeros(997)

for i in range(998):
    cumulative      = np.sum(M_lost[:i])
    N_mmgrain       = cumulative/m_bins[53]
    surface_area    = N_mmgrain * 4 * np.pi * (r_bins[53] ** 2)
    total_surface   = np.sum(step_areas[:i])
    fractional_area = np.abs(surface_area/total_surface)
    
    cumulative_lost[i]     = cumulative
    N_mmgrain_cum[i]       = cumulative/m_bins[53]
    surface_area_cum[i]    = surface_area
    fractional_area_cum[i] = fractional_area
    
# 2. Time-evolution studies -----------------------------------------------------
    
N_mmgrain = M_lost/m_bins[53]
surface_area = N_mmgrain * 4 * np.pi * (r_bins[53] ** 2)
total_surface = step_areas
fractional_area = np.abs(surface_area/step_areas[1:997])

In [ ]:
###############################
# SURFACE AREA LOST PLOTS
###############################
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), sharex=True)
plt.subplots_adjust(hspace=.2)

ax[0,0].plot(M_lost[:997]*(-1), linewidth = 3, color = "#205493")
ax[0,0].set_title(r"$M_{tot}$ (g)", fontsize=15)

ax[0,1].plot(N_mmgrain[:997]*(-1), linewidth = 3, color = "#205493")
ax[0,1].set_title(r"$N_{mm}$", fontsize=15)

ax[1,0].plot(surface_area[:997]*(-1), linewidth = 3, color = "#205493")
ax[1,0].set_title(r"S (cm$^{2}$)", fontsize=15)

ax[1,1].plot(fractional_area, linewidth = 3, color = "#205493")
ax[1,1].set_title(r"$\Delta$ H$_2$O", fontsize=15)

for i in range(2):
    ax[1,i].set_xlabel(r"N$_{transits}$", fontsize=15)
    for j in range(2):
    
        ax[j,i].tick_params(axis='x', labelsize=12)
        ax[j,i].tick_params(axis='y', labelsize=12)

plt.savefig( f"{plot_path}surface_areas.pdf", bbox_inches='tight')

In [ ]:
##############################
# CHECK CONSERVATION OF MASS #
##############################

# Here I convert the number densities back to mass densities, then compare initial setup to that after the grain growth (final timestep)


# Define function to convert number density back to mass density
def n_to_rho(n_dens_array):
    rho_dust = np.zeros(shape=(50, 100))
    for i in (range(50)):
        for j in range(100):
            rho_cell = 0
            for k in range(n_bins):
                rho_bin = n_dens_array[i][j][k] * m_bins[k]
                rho_cell += rho_bin
            rho_dust[i][j] = rho_cell
    return rho_dust


# Populate arrays
initial_rho = n_to_rho(n_density_initial)       # density at the start
end_rho     = n_to_rho(n_densities_evolution)   # density at the end

initial_rho[initial_rho < 1e-35] = 1e-35
end_rho[end_rho < 1e-35] = 1e-35
# Setup the figure
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10.5, 4), sharey=True)

rcoord = np.logspace(np.log10(1), 2.5, num=100)
zrcoord = np.linspace(0,.5, 50)
r, zr  = np.meshgrid(rcoord, zrcoord)

levels_rho = np.arange(-20, -8, 1)

initial_rho  = ax[0].contourf(r, zr, np.log10(initial_rho), levels=levels_rho, cmap='inferno', extend='both', vmin = -20)
end_rho = ax[1].contourf(r, zr, np.log10(end_rho), levels=levels_rho, cmap='inferno', extend='both')


ax[0].annotate('$\\rho_\\mathrm{dust}$ (START)', (2, 0.4), c='white', fontsize=22)
ax[1].annotate('$\\rho_\\mathrm{dust}$ (END)', (2, 0.4), c='white', fontsize=22)

for i in range(2):
    ax[i].set_xscale('log')
    ax[i].set_xlabel('r (AU)', fontsize=15)
    ax[i].tick_params(axis='x', labelsize=12)
    ax[i].tick_params(axis='y', labelsize=12)
ax[0].set_ylabel('z/r', fontsize=15)
plt.tight_layout()

cbar_ax = fig.add_axes([0.05, -0.05, .95, 0.05])  # [left, bottom, width, height]
cbar = fig.colorbar(end_rho, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r'$\rho_\mathrm{dust}$ (g/cm$^{3}$)',fontsize=18)
cbar.set_ticks([-19,-17,-15,-13,-11,-9])
cbar.set_ticklabels([r'$10^{-19}$',r'$10^{-17}$',r'$10^{-15}$',r'$10^{-13}$',r'$10^{-11}$',r'$10^{-9}$'], fontsize = 12)  # Set your own tick values here
plt.savefig(plot_path+"rho_dust_evolution.pdf", bbox_inches='tight')
plt.show()

In [ ]:
###########################################################
# PLOT DUST DENSITIES                                     #
###########################################################

fig, axes = plt.subplots(nrows=5, ncols=11, figsize=(20, 10), sharex=True, sharey=True)
for (i, ax) in enumerate(fig.axes):
    
    if i < 54:
    
        z_data_0 = np.log10(n_densities_evolution[:,:,i])
        z_data_0[z_data_0<-75] = -75
        im = ax.contourf(r, zr, z_data_0, levels=np.arange(-15, 2, 0.5), cmap='inferno', extend='both')
        
        ax.set_xscale('log')
        ax.annotate(f'{r_bins[i]*1e4:.1e} $\mu$m', (2, 0.4), fontsize=12, c='white')
        
    else:
        ax.axis('off')


plt.tight_layout()
cbar_ax = fig.add_axes([1.02, 0., 0.02, 1])  # [left, bottom, width, height]
cbar = fig.colorbar(im, cax=cbar_ax, )
cbar.set_label(r'$n_\mathrm{dust}$ (cm$^{-3}$)',fontsize=18)

fig.text(0.5, -0.04, 'r (AU)', ha='center', va='center',fontsize=22)
fig.text(-0.04, 0.5, 'z/r', ha='center', va='center',fontsize=22,  rotation='vertical')
plt.savefig(plot_path+"n_densities_evolution.pdf" ,bbox_inches='tight')
plt.show()

# Compare to the initial distribution...
fig, ax = plt.subplots(nrows=5, ncols=11, figsize=(20, 10), sharex=True, sharey=True)



for (i, ax) in enumerate(fig.axes):
    
    if i < 54:
    
        z_data_0 = np.log10(n_density_initial[:,:,i])        
        z_data_0[z_data_0<-35] = -35
        ax.contourf(r, zr, z_data_0, levels=np.arange(-10, 2, 0.5), cmap='inferno', extend='both')
        
        ax.set_xscale('log')
        ax.annotate(f'{r_bins[i]*1e4:.1e} $\mu$m', (2, 0.4), fontsize=12, c = 'white')
        
    else:
        ax.axis('off')
        
plt.tight_layout()
cbar_ax = fig.add_axes([1.02, 0., 0.02, 1])  # [left, bottom, width, height]
cbar = fig.colorbar(im, cax=cbar_ax, )
cbar.set_label(r'$n_\mathrm{dust}$ (cm$^{-3}$)',fontsize=18)
fig.text(0.5, -0.04, 'r (AU)', ha='center', va='center',fontsize=22)
fig.text(-0.02, 0.5, 'z/r', ha='center', va='center',fontsize=22,  rotation='vertical')
plt.savefig(plot_path + "n_density_initial.pdf",bbox_inches='tight')

plt.show()

## 3. Effects on Chemistry. Long term evolution of C/O.


In [ ]:
water_denisty_evolution = np.zeros(1447)

water = np.median(water_density[snowline_mask])
for i in range(997):
    water_denisty_evolution[i] = water * ((1 - fractional_area[i]))
    water = water_denisty_evolution[i]

CtoOmap = np.median(co2_density[snowline_mask])/water_denisty_evolution
CtoOmap_const = np.median(co2_density[snowline_mask])/(np.median(water_density[snowline_mask])*(1-2e-13)**3.5e12)

In [ ]:
timeticks = (np.linspace(0,1000, 5) *28.9)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))


ax.plot((CtoOmap-0.5)*1e9, linewidth = 3,  color = "#205493")


ax.set_ylabel(r"$\Delta$(C/O) · 10$^{-9}$ ",fontsize=18)
ax.set_xlabel(r"N$_{transits}$", fontsize=18)
ax.text(560, 0.2, r"$\Delta t$ = 0.016 Myrs", rotation=90, va='center', )

plt.axvline(550, linestyle = "--", linewidth = 3, color = "#205493", alpha = .5,)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig(plot_path + "CtoO_evolution.pdf",bbox_inches='tight')
plt.show()